In [ ]:
import joblib
import click
import json
import time
import os
import itertools
import collections.abc
from shutil import copy,SameFileError
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
# from qiskit_ibm_provider import IBMProvider

from quantum.Quantum import QuantumRegressor
from quantum.Evaluate import evaluate
from settings import ANSATZ_LIST, ENCODER_LIST
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import InconsistentVersionWarning

In [ ]:


# device='qiskit.aer'
# device_backend = FakeCairoV2()
# backend = AerSimulator.from_backend(device_backend)
# device = qml.device(device, wires=num_qubits, backend=backend, shots=shots)


device='qulacs.simulator'
num_qubits=5
device = qml.device(device, wires=num_qubits)


In [ ]:
def circuit(features, parameters):
    #  builds the circuit with the given encoder and variational circuits.
    #  encoder and variational circuits must have only two required parameters, params/feats and wires
    for i in range(re_upload_depth):
        params = initial_parameters.reshape(re_upload_depth,-1)[i]
        
        encoder(features, wires=range(num_qubits))
        variational(params, wires=range(num_qubits))

    return qml.expval(qml.PauliZ(0))

In [ ]:
def get_depth(enc,var,rud,features,parameters):
    circuit_gates=0
    circuit_depth=0
    for i in range(rud):
        params = initial_parameters.reshape(rud,-1)[i]
        
        encoder_specs=qml.specs(qml.QNode(encoder,device,expansion_strategy='device'))(features, wires=range(num_qubits))['resources']
        var_specs=qml.specs(qml.QNode(variational,device,expansion_strategy='device'))(params, wires=range(num_qubits))['resources']

        circuit_gates+=var_specs.num_gates + encoder_specs.num_gates
        circuit_depth+=var_specs.depth + encoder_specs.depth
    return circuit_gates,circuit_depth

In [ ]:
flat_results = []
for i in glob('../qml_DDCC/5/*/*results.json'):
    p = os.path.basename(os.path.dirname(i))
    with open(i,'r') as f:
        og_results=json.load(f)
    ENCODER, ANSATZ = p.split('_')
    # print(ENCODER, ANSATZ)
    encoder = ENCODER_LIST[ENCODER]
    variational=ANSATZ_LIST[ANSATZ]

    LAYERS=1
    re_upload_depth=1
    variational.layers = LAYERS
    variational.set_wires(range(num_qubits))
    

    num_params = variational.num_params * re_upload_depth
    generator = np.random.default_rng(12958234)
    initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
    feat = generator.uniform(-1, 1, num_qubits)
    
    qnode = qml.QNode(circuit, device)   

    # print(f"{p} AL_{LAYERS} RUD_{re_upload_depth}\n",circuit_specs)
    # qml.draw_mpl(qnode,level=1)(feat,initial_parameters)
    # plt.show()
    
    circuit_specs=qml.specs(qnode)(feat,initial_parameters)['resources']

    circuit_gates,circuit_depth= get_depth(encoder,variational,1,feat,initial_parameters)
    
    with open(os.path.join(f"../qml_DDCC/5/{p}",'model_log.csv'),'r') as f:
        # Create df
        timedf = pd.DataFrame(pd.to_datetime([i.split(',')[0] for i in f.readlines() if '[' in i], format='%a %b %d %H:%M:%S %Y'),columns=['Time'])
        # Get time between iterations and get the mean time
        timedf['Iter time']=[i.total_seconds() for i in timedf['Time'].diff()]
    
    flat_results.append((1,1,p,og_results['MAE_train'],og_results['MAE_test'],circuit_gates,circuit_depth,timedf['Iter time'].mean()))

In [ ]:
df = pd.DataFrame(flat_results,columns=['AL','RUD','pairs','MAE Train','MAE Test','num_gates','depth','mean_iter'])

In [ ]:
df[["MAE Train","MAE Test"]].describe()

In [ ]:
palette=sns.color_palette('Paired',4)

In [ ]:
median_Train,median_Test=np.median(df[['MAE Train',"MAE Test"]].values,axis=0)

In [ ]:
sns.histplot(data=df,y='mean_iter')

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 16
BIGGER_SIZE = 12

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

fig, ax = plt.subplots(1,2,sharey=True,figsize=(10,5))
sns.scatterplot(df,x='MAE Train',y='mean_iter',label='Train',ax=ax[0],color=palette[0])
sns.scatterplot(df,x='MAE Test',y='mean_iter',label='Test',ax=ax[0],color=palette[1])
# plt.hlines(df.describe().loc['mean','mean_iter'],0,50)
# ax[0].vlines(median_Train,0,700,color='r')
# ax[0].vlines(median_Test,0,700,color='r')

ax[0].set_xlim(0,4e-2)
ax[0].set_ylim(0,150)
ax[0].set_ylabel('Mean Time/Training Iteration (s)')
ax[0].set_xlabel('MAE (E$_{h}$)')
ax[0].legend()

ax[1].scatter(df['depth'],df['mean_iter'],color=palette[3])
ax[1].set_xlim(0,50)
ax[1].set_xlabel('Circuit Depth')
plt.tight_layout()
plt.savefig('AllDDCC5_circuitdepth_MAE_vs_iterationtime.png',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
# with open('kill_list.txt','w') as g:
#     for i in df[df['mean_iter']>100]['pairs'].values:
#         g.write(i+'\n')

In [ ]:
np.mean(df['mean_iter'])

In [ ]:
# for i in glob('../BSE_RUD_AL_test/16qubit/*'):
#     for j in df[df['mean_iter']>100]['pairs'].values:
#         if os.path.exists(os.path.join(i,j)):
#             print(i,j)
#             print(df[df['pairs']==j])